# Normalize & compute stats

In [1]:
import submitit
import scanpy as sc 
import os
import numpy as np
import pickle as pkl
from functools import partial
from scipy import stats


def compute_stats(row, metadata):
    row_rep = [rr.replace("/", "-") for rr in row]
    if os.path.exists(f"../DISCOtmp/features/{'_'.join(row_rep)}_normalized.pkl"):
        pass
    else:
        samples = metadata.loc[tuple(row), "sampleId"].unique()
        X = []
        for sample in samples:
            if os.path.exists(f"../../scrna/DISCOtmp/{sample}.h5ad"):
                adata = sc.read_h5ad(f"../../scrna/DISCOtmp/{sample}.h5ad")
                sc.pp.normalize_total(adata, target_sum = 1e4)
                X.append(adata.X[adata.obs["cell.type"] == row[-1]].A)
        X = np.concatenate(X, axis=0)
        # X = sp.vstack(X)
        
        freq = np.count_nonzero(X, axis=0, keepdims=True) / len(X)
        mean = X.mean(axis=0, keepdims=True)
        # std = X.std(axis=0)
        skew = stats.skew(X, axis=0, keepdims=True)
        kurtosis = stats.kurtosis(X, axis=0, keepdims=True)
        
        X[X == 0.] = np.nan
        nonzero_mean = np.nanmean(X, axis=0, keepdims=True)
        # nonzero_std = np.nanstd(X, axis=0)
        nonzero_skew = stats.skew(X, axis=0, nan_policy="omit", keepdims=True)
        nonzero_kurtosis = stats.kurtosis(X, axis=0, nan_policy="omit", keepdims=True)
        
        with open(f"../DISCOtmp/features/{'_'.join(row_rep)}_normalized.pkl", "wb") as f:
            pkl.dump([freq, mean, skew, kurtosis, nonzero_mean, nonzero_skew, nonzero_kurtosis], f)
            
        
def submit_job(common_indices, metadata):
    executor = submitit.AutoExecutor(folder="slurm_logs")
    executor.update_parameters(slurm_partition="cpu256",
                            slurm_job_name="scrna",
                            cpus_per_task=8,
                            timeout_min=6000,
                            # slurm_array_parallelism=20,
                            )
    jobs = executor.map_array(partial(compute_stats, metadata=metadata), common_indices.to_numpy().tolist())

ModuleNotFoundError: No module named 'submitit'

In [2]:
def get_metadata(tumor: bool, num_cells: int):
    if os.path.exists("../../scrna/DISCOtmp/metadata_" + ("tumor" if tumor else "normal") + ".pkl"):
        with open("../../scrna/DISCOtmp/metadata_" + ("tumor" if tumor else "normal") + ".pkl", "rb") as f:
            metadata = pkl.load(f)
    else:
        print("error")
        # ft = dt.Filter(sample_type=["Primary Tumor", "Cancer"] if tumor else ["Normal"])
        # metadata = dt.filter_disco_metadata(ft)
        # with open("../../scrna/DISCOtmp/metadata_" + ("tumor" if tumor else "normal") + ".pkl", "wb") as f:
        #     pkl.dump(metadata, f)
    
    sample_metadata = metadata.sample_metadata
    sample_metadata = sample_metadata[(sample_metadata["treatment"].isna()) & (sample_metadata["diseaseStage"].isna())]
    if tumor:
        sample_metadata = sample_metadata[~sample_metadata["diseaseSubtype"].isin(["pre-neoplastic", "metastatic breast cancer"])]
        sample_metadata = sample_metadata[~sample_metadata["disease"].isin(["COVID-19", "Juxta tumor", "intraductal papillary mucinous neoplasm", "lipoma"])]
        sample_metadata = sample_metadata[~sample_metadata["disease"].isna()]
    else:
        sample_metadata = sample_metadata[sample_metadata["disease"].isna()]
    
    cm = metadata.cell_type_metadata
    df_metadata = sample_metadata.merge(cm)
    df_metadata = df_metadata.replace({"esophagus squamous cell carcinoma": "esophageal squamous cell carcinoma",
                                       "nasopharyngeal tumor": "nasopharyngeal carcinoma",
                                       "pancreatic adenocarcenoma": "pancreatic ductal adenocarcinoma",
                                       "renal adenocarcinoma": "renal cell carcinoma"})
    
    if tumor:
        df_cellnum = df_metadata[["disease", "tissue", "cellType", "cellNumber"]].groupby(by=["disease", "tissue", "cellType"]).sum()
    else:
        df_cellnum = df_metadata[["tissue", "cellType", "cellNumber"]].groupby(by=["tissue", "cellType"]).sum()
    indices = df_cellnum[df_cellnum["cellNumber"] >= num_cells].drop("cellNumber", axis=1).reset_index()
    print(indices)
    
    return df_metadata, indices

In [4]:
tumor_metadata, tumor_indices = get_metadata(tumor=True, num_cells=500)
normal_metadata, normal_indices = get_metadata(tumor=False, num_cells=500)

tumor_common_indices = tumor_indices.merge(normal_indices)
normal_common_indices = tumor_common_indices.drop("disease", axis=1).drop_duplicates()
tumor_metadata = tumor_metadata.merge(tumor_common_indices).set_index(["disease", "tissue", "cellType"])
normal_metadata = normal_metadata.merge(normal_common_indices).set_index(["tissue", "cellType"])

tumor_metadata["sampleId"].drop_duplicates().to_csv("../supplementary/file1_tumor_samples_normalize_all.csv", index=False)
normal_metadata["sampleId"].drop_duplicates().to_csv("../supplementary/file1_normal_samples_normalize_all.csv", index=False)

                        disease           tissue           cellType
0     Cushing's disease adenoma  pituitary gland         Delta cell
1     Cushing's disease adenoma  pituitary gland     Endocrine cell
2     Cushing's disease adenoma  pituitary gland  Memory CD4 T cell
3     Cushing's disease adenoma  pituitary gland   Naive CD4 T cell
4     Cushing's disease adenoma  pituitary gland           Pericyte
...                         ...              ...                ...
1086             uveal melanoma              eye         CD4 T cell
1087             uveal melanoma              eye         CD8 T cell
1088             uveal melanoma              eye    GZMB CD8 T cell
1089             uveal melanoma              eye         Melanocyte
1090             uveal melanoma              eye   Naive CD8 T cell

[1091 rows x 3 columns]
            tissue                     cellType
0          adipose                    Adipocyte
1          adipose                       B cell
2          adip

In [7]:
from tqdm import tqdm
import warnings
import scipy.sparse as sp

warnings.filterwarnings(action="ignore")

samples = normal_metadata.loc["blood", "Naive CD4 T cell"]["sampleId"].unique()
X = []
for sample in tqdm(samples):
    if os.path.exists(f"../../scrna/DISCOtmp/{sample}.h5ad"):
        print(sample)
        adata = sc.read_h5ad(f"../../scrna/DISCOtmp/{sample}.h5ad")
        sc.pp.normalize_total(adata, target_sum = 1e4)
        X.append(adata.X[adata.obs["cell.type"] == "Naive CD4 T cell"])
    else:
        print(("fail", sample))
# X = np.concatenate(X, axis=0)
X = sp.vstack(X).A

freq = np.count_nonzero(X, axis=0, keepdims=True) / len(X)
mean = X.mean(axis=0, keepdims=True)
# std = X.std(axis=0)
skew = stats.skew(X, axis=0, keepdims=True)
kurtosis = stats.kurtosis(X, axis=0, keepdims=True)

X[X == 0.] = np.nan
nonzero_mean = np.nanmean(X, axis=0, keepdims=True)
# nonzero_std = np.nanstd(X, axis=0)
nonzero_skew = stats.skew(X, axis=0, nan_policy="omit", keepdims=True)
nonzero_kurtosis = stats.kurtosis(X, axis=0, nan_policy="omit", keepdims=True)

with open(f"../DISCOtmp/features/blood_Naive CD4 T cell_normalized.pkl", "wb") as f:
    pkl.dump([freq, mean, skew, kurtosis, nonzero_mean, nonzero_skew, nonzero_kurtosis], f)

  0%|          | 0/673 [00:00<?, ?it/s]

B1_AA220907_HC


  0%|          | 1/673 [00:00<05:51,  1.91it/s]

B1_CHI014_HC


  0%|          | 2/673 [00:01<06:13,  1.80it/s]

...

SGI_H4


100%|█████████▉| 672/673 [15:17<00:01,  1.37s/it]

SIGAH12


100%|██████████| 673/673 [15:18<00:00,  1.36s/it]


KeyboardInterrupt: 

In [4]:
submit_job(tumor_common_indices, tumor_metadata)
submit_job(normal_common_indices, normal_metadata)

KeyboardInterrupt: 

# Preprocess cell type data

In [2]:
import gzip
import os
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm

import scanpy as sc

BASE_DIR = "../data"
num_cells = 500

def get_features(metadata, common_indices):
    # common_indices = common_indices[~((common_indices["tissue"] == "blood") & (common_indices["cellType"].isin(["Memory CD4 T cell", "Naive CD4 T cell", "CD14 monocyte", "CD16 NK cell", "GZMB CD8 T cell", "Naive CD8 T cell"])))]
    results = []
    adata = sc.read_h5ad(f"../../scrna/DISCOtmp/AML003_3p.h5ad")
    for _, row in tqdm(common_indices.iterrows(), total=len(common_indices)):
        row_values = [rv.replace("/", "-") for rv in row.values]
        if os.path.exists(f"../DISCOtmp/features/{'_'.join(row_values)}_normalized.pkl"):
            with open(f"../DISCOtmp/features/{'_'.join(row_values)}_normalized.pkl", "rb") as f:
                results.append(pkl.load(f))
        else:
            print(row)
    
    print(len(results), len(common_indices))
    freq, mean, skew, kurtosis, nonzero_mean, nonzero_skew, nonzero_kurtosis = list(zip(*results))

    return {
        "freq": pd.DataFrame(np.concatenate([np.concatenate(freq, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "mean": pd.DataFrame(np.concatenate([np.concatenate(mean, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "skew": pd.DataFrame(np.concatenate([np.concatenate(skew, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "kurtosis": pd.DataFrame(np.concatenate([np.concatenate(kurtosis, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "nonzero_mean": pd.DataFrame(np.concatenate([np.concatenate(nonzero_mean, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "nonzero_skew": pd.DataFrame(np.concatenate([np.concatenate(nonzero_skew, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
        "nonzero_kurtosis": pd.DataFrame(np.concatenate([np.concatenate(nonzero_kurtosis, axis=0), common_indices], axis=1), columns=adata.var.index.tolist() + common_indices.columns.tolist()),
    }

    # print(len(results), len(common_indices))

    # return pd.DataFrame(np.concatenate([np.concatenate(results, axis=0), common_indices], axis=1), 
    #                     columns=adata.var.index.tolist() + common_indices.columns.tolist())

if os.path.exists(f"{BASE_DIR}/preprocessed/tumor_{num_cells}_normalized_all.pkl.gz"):
    with gzip.open(f"{BASE_DIR}/preprocessed/tumor_{num_cells}_normalized_all.pkl.gz", "rb") as f:
        tumor_feature = pkl.load(f)
else:
    tumor_feature = get_features(tumor_metadata, tumor_common_indices)
    with gzip.open(f"{BASE_DIR}/preprocessed/tumor_{num_cells}_normalized_all.pkl.gz", "wb") as f:
        pkl.dump(tumor_feature, f)

if os.path.exists(f"{BASE_DIR}/preprocessed/normal_{num_cells}_normalized_all.pkl.gz"):
    with gzip.open(f"{BASE_DIR}/preprocessed/normal_{num_cells}_normalized_all.pkl.gz", "rb") as f:
        normal_feature = pkl.load(f)
else:
    normal_feature = get_features(normal_metadata, normal_common_indices)
    with gzip.open(f"{BASE_DIR}/preprocessed/normal_{num_cells}_normalized_all.pkl.gz", "wb") as f:
        pkl.dump(normal_feature, f)

In [3]:
feature_names = tumor_feature["freq"][["disease", "tissue", "cellType"]].values
# tumor_feature["freq"][["disease", "tissue", "cellType"]].to_csv("../../scrna/supplementary/disease_tissue_celltype_normalize_all.csv")
tumor_feature["freq"][["disease", "tissue", "cellType"]].to_csv("../supplementary/file1_disease_tissue_celltype_normalize_all.csv", index=False)
